In [1]:
from math import asin
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

## Урок 8. Python применение статистических критериев на практике - пайплайн оценки A/B
На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [2]:
df = pd.read_excel('gb_sem_ab_testing-master/gb_sem_ab_testing-master/data/gb_sem_8_hm.xlsx')
df.head(5)

C:\Users\Acer\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [3]:
df.shape


(10000, 3)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [5]:
# группы распределены поровну
df.groupby('VARIANT_NAME').count()

,USER_ID,REVENUE
VARIANT_NAME,,
control,4984,4984
variant,5016,5016


In [6]:
df.agg({'REVENUE': 'max'}), df.agg({'REVENUE': 'min'}), 

(REVENUE    196.01
 dtype: float64,
 REVENUE    0.0
 dtype: float64)

In [7]:
# смотрим сколько уникальных пользователей, для понимая задвоения и как разбились группы
df.USER_ID.nunique()
# видим, что ID повторяются, значит их надо сгруппировать или убрать, если они в обоих группах

6324

In [8]:
df.VARIANT_NAME.nunique()

2

In [9]:
df.REVENUE.sum()

994.47

In [10]:
# группируем всех юзеров по их группам (в т.ч. одинаковых покупателей) 
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})
df.shape

(7865, 3)

In [92]:
# посмотрим, как распределены группы
df.groupby('VARIANT_NAME').count()

,USER_ID,REVENUE
VARIANT_NAME,,
control,3931,3931
variant,3934,3934


In [12]:
# как аналог df.USER_ID.nunique()
df.groupby('USER_ID', as_index=False).count()

,USER_ID,VARIANT_NAME,REVENUE
0,2,1,1
1,3,2,2
2,4,1,1
3,5,1,1
4,6,1,1
...,...,...,...
6319,9993,1,1
6320,9995,1,1
6321,9996,2,2
6322,9998,1,1


In [25]:
# убираем задвоившихся пользователей
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [31]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )
len(unique_ids)

4783

In [88]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)
df_new.shape

(4783, 3)

In [59]:
df_new.describe()

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [93]:
df_new.head()

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.0
3,4,variant,0.0
4,5,variant,0.0
5,6,variant,0.0
6,9,variant,0.0


In [94]:
# проверим, Как распределены группы
df_new.groupby('VARIANT_NAME').count()

,USER_ID,REVENUE
VARIANT_NAME,,
control,2390,2390
variant,2393,2393


# сплитование прошло равномерно

In [ ]:
#pip install plotly
import plotly.express as px

In [96]:
fig = px.histogram(df_new,
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   title='revenue on user',
                   marginal = 'box',
                   nbins = 100)

fig.show()

In [89]:
fig = px.histogram(df_new, x="REVENUE",
                   color='VARIANT_NAME', barmode='group',
                   height=400)
fig.show()

In [99]:
#выделяем обе группы для сравнения (проведения теста). ТЕСТ  количественной метрики
control2 = df_new[df_new['VARIANT_NAME'] == 'control'].copy(deep=True)
test2 = df_new[df_new['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [80]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    
    # int = 10_000 количество итераций теста - чем выше, тем точнее можно определить равномерность распределения
    # берется исходя из опыта его применения в конкретных случаях
    
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [90]:
continious_result(control, test, column='REVENUE')

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:08<00:00, 1194.50it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.028393,0.205962,0.757808,"[-0.023, 0.205]",0.07429


### Вывод
1. alpha > 0.5, beta = 0.75. Получили вероятность больше 0,05, очень высокая мощность теста, и небольшая разница между 
средними выборок. 0 попадает в доверительный интервал 
2. Делаем вывод, что у нас нет статистически значимых различий в группах.
3. Принимает нулевые гипотезу (что между группами нет различий)
4. Таким образом, увеличить доход, используя эту гипотезу - не получится. Изменения не вводить